In [10]:
# Dependencies
import tweepy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from datetime import datetime
import matplotlib
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [11]:
consumer_key = "EJIuSEcMCdhvIZ7iaXQYbjLUY"
consumer_secret = "Cdxt84aPBXQFmKL5ZEwOC5MquM4BvzfqSk7gjMzRe4X15rviFw"
access_token = "983862551497723904-ClTOfKgpd9Gc5nhfYL61fc9ssKvZLhS"
access_token_secret = "7GhbOxlludczquMzYANZm2gDvMIHLfa09tnMf9sdD9gZy"

In [12]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,parser=tweepy.parsers.JSONParser())

In [24]:
lang = "en"
target="PlotBot Analyze"
#tuple to hold to stop repeating
Tuple_screenname_tweettarget=[]


def sentiment_analyse():
    appended_results=[]
    counter=-1
    oldest_tweet=None
    
    
    
    #count=1, it returns only one search
    public_tweets = api.search(target, count=1,result_type="recent")
    for tweet in public_tweets['statuses']:
        
        print(tweet)
        tweet_id = tweet["id"]
        tweet_author=tweet['user']['screen_name'] 
        raw_text=tweet["text"]
        raw_text_split=raw_text.split(":")
        keyword_to_analyze=raw_text_split[1]
        
        if (tweet_author,keyword_to_analyze) not in Tuple_screenname_tweettarget:
            
            Tuple_screenname_tweettarget_tuple=(tweet_author,keyword_to_analyze )
            Tuple_screenname_tweettarget.append(Tuple_screenname_tweettarget_tuple)
            
            for x in range(5):
                tweets_keyword = api.search(keyword_to_analyze, max_id=oldest_tweet,count=100,lang=lang)
            
                for tweet_1 in tweets_keyword["statuses"]:
                    tweet_text=tweet_1['text']
                    
                    vader_analysis = analyzer.polarity_scores(tweet_text)
                    compound=vader_analysis['compound']
                    positive=vader_analysis['pos']
                    negative=vader_analysis['neg']
                    neutral=vader_analysis['neu']
                    
                    results = {
                   "Date":tweet['created_at'], 
                   "Keyword": keyword_to_analyze,
                   "Compound Score": compound,
                   "Postive Score": positive,
                   "Neutral Score": neutral,
                   "Negative Score": negative,
                   "Tweets Ago":counter
                    }
                    counter=counter-1
                    appended_results.append(results)
                    oldest_tweet=tweet_1['id']-1
        
            analysis_plot(appended_results)
            #api.update_with_media("Analysis.png",f"New Tweet Analysis: {keyword_to_analyze}(Requested by @{tweet_author})",in_reply_to_status_id=tweet_id)
    
        elif (tweet_author,keyword_to_analyze) in Tuple_screenname_tweettarget:
            print("Already analyzed")
    
    #unless u mention the user name with @ symbol, it doesnt refelect in replying    

In [27]:
def analysis_plot(appended_results):
    
    df=pd.DataFrame(appended_results)
    y_axis=df["Compound Score"]
    x_axis=df["Tweets Ago"]
    x_axis_max=np.max(x_axis)
    x_axis_min=np.min(x_axis)
    
    plt.plot(x_axis,y_axis,marker='o', linewidth=0.5,label="Tweets" + "\n"+ df['Keyword'][2],color="brown")
    plt.legend(loc='upper left', bbox_to_anchor=(1, 0.9),mode="Extended")
    now=datetime.now()
    now=now.strftime("%Y-%m-%d %H:%M")
    plt.xlim(x_axis_min, x_axis_max)
    plt.xlabel("Tweets Ago")
    plt.ylabel("Tweet Polarity")
    plt.title(f"Sentiment Analysis of tweets ({now})")
    plt.grid()
    plt.savefig("Analysis.png", bbox_inches='tight')
    

In [28]:
counter_2= 0
while(True):
    sentiment_analyse()
    counter_2 = counter_2 + 1
    time.sleep(300)
    

{'created_at': 'Tue Apr 10 21:30:14 +0000 2018', 'id': 983819470815739904, 'id_str': '983819470815739904', 'text': '@PlotBot Analyze: @CNN', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'plotbot', 'name': 'Plotbot', 'id': 17582138, 'id_str': '17582138', 'indices': [0, 8]}, {'screen_name': 'CNN', 'name': 'CNN', 'id': 759251, 'id_str': '759251', 'indices': [18, 22]}], 'urls': []}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': 17582138, 'in_reply_to_user_id_str': '17582138', 'in_reply_to_screen_name': 'plotbot', 'user': {'id': 13293552, 'id_str': '13293552', 'name': 'Cha Lee', 'screen_name': 'Chuckde3zy', 'location': '', 'description': "People never get the flowers while they could still smell 'em.", 'url': None, 'entities': {'description': {'urls': [